### Importing

In [1]:
from CB_IPO import scrape

### Initializing

In [2]:
scraper = scrape()

### Scraping for recent IPO filings (S-1)

In [21]:
company_names, file_dates, form_types = scraper.edgar_scrape(5)
print(company_names)
print(file_dates[1:])
print(form_types)


['Intuitive Machines, Inc.  (LUNR, LUNRW) ', 'Nuburu, Inc.  (BURU, BURU-WT) ', 'SOLIGENIX, INC.  (SNGX) ', 'NovaBay Pharmaceuticals, Inc.  (NBY) ', 'Kodiak Gas Services, Inc. ']
['2023-03-31', '2023-03-31', '2023-03-31', '2023-03-31']
{'S-1', 'S-1/A'}


### Formatting into Dataframe

In [15]:
df = scraper.generate_df(6,1)
df.iloc[1:]

names filing date
1                     Nuburu, Inc.  (BURU, BURU-WT)   2023-03-31
2                           SOLIGENIX, INC.  (SNGX)   2023-03-31
3              NovaBay Pharmaceuticals, Inc.  (NBY)   2023-03-31
4                         Kodiak Gas Services, Inc.   2023-03-31
5  Artificial Intelligence Technology Solutions I...  2023-03-31

### Modifying Search Dates

In [14]:
scraper.set_search_date("2021-01-01", "2023-03-31")

company_names, file_dates, form_types = scraper.edgar_scrape(5)

df = scraper.generate_df(6,1)
df.iloc[1:]

names filing date
1                     Nuburu, Inc.  (BURU, BURU-WT)   2023-03-31
2                           SOLIGENIX, INC.  (SNGX)   2023-03-31
3              NovaBay Pharmaceuticals, Inc.  (NBY)   2023-03-31
4                         Kodiak Gas Services, Inc.   2023-03-31
5  Artificial Intelligence Technology Solutions I...  2023-03-31

### Modifying Form Types

In [13]:
scraper.reset_url()
scraper.set_search_date("2021-12-30", "2022-01-01")
scraper.add_forms(['10-K', '10-Q'])

company_names, file_dates, form_types = scraper.edgar_scrape(5)
print (form_types)

df = scraper.generate_df(6,1)
df.iloc[1:]

{'10-Q', '10-Q/A', '10-K'}


names filing date
1                        GOLDEN GRAIN ENERGY   2021-12-30
2  Financial Gravity Companies, Inc.  (FGCO)   2021-12-30
3                        NEOGEN CORP  (NEOG)   2021-12-30
4                EASTON PHARMACEUTICALS INC.   2021-12-30
5              Nestbuilder.com Corp.  (NBLD)   2021-12-30

### Generating Reference Codes and Accession Numbers

In [4]:
#This is the cik that identifies Pfizer
cik = '0000078003'

#This will yield all references and the name of a company tied to a cik
references, name = scraper.get_refs(cik,3)

#This will yield all accession numbers tied to a cik
accession_numbers = scraper.get_anums(cik,3)

print(accession_numbers)
print(references)
print(name)

['000007800323000024', '000007800322000027', '000007800321000038']
['pfe-20221231.htm', 'pfe-20211231.htm', 'pfe-20201231.htm']
PFIZER INC  (PFE) 


### Finding xbrl Links for a Company

In [5]:
links,c_name = scraper.create_links(cik, 3)

for link in links:
    print(link)

https://www.sec.gov/ix?doc=/Archives/edgar/data/0000078003/000007800323000024/pfe-20221231.htm
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000078003/000007800322000027/pfe-20211231.htm
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000078003/000007800321000038/pfe-20201231.htm


### Scraping Link for Financial Info and Calculating Ratio

In [10]:
link = links[0]

Financials = scraper.scrape_xbrl(link)
for f in Financials.items():
    print(''+f[0]+': '+str(f[1]))
print()

ratios = scraper.calculate_ratios(Financials)

for f in ratios.items():
    print(''+f[0]+': '+str(f[1]))
print()

Total Assets: 197205.0
Total Liabilities: 101288.0
Total Equity: 95916.0
Current Assets: 51259.0
Current Liabilities: 42138
Net Income: 31372.0
Long Term Debt: 32884.0
Current Debt: 2945.0
Inventory: 8981.0
Registrant: PFIZER INC

D/E: 1.0560073397556196
ROE: 0.3270778597939864
Working Capital: 1.216455455883051
Quick: 1.0033224168209218
TD/TA: 0.181684034380467
ROA: 0.1590831875459547



### 10-K Dataframe Summary

In [16]:
raw_df = scraper.summarize_10k(link, flag = 'raw')
ratio_df = scraper.summarize_10k(link, flag = 'liquidity')
lev_df = scraper.summarize_10k(link, flag = 'leverage')

display(raw_df)
print()
display(ratio_df)
print()
display(lev_df)


Account      Amount
0         Total Assets    197205.0
1    Total Liabilities    101288.0
2         Total Equity     95916.0
3       Current Assets     51259.0
4  Current Liabilities       42138
5           Net Income     31372.0
6       Long Term Debt     32884.0
7         Current Debt      2945.0
8            Inventory      8981.0
9           Registrant  PFIZER INC

Ratio     Value
0            Quick  1.003322
1  Working Capital  1.216455

Ratio     Value
0    D/E  1.056007
1  TD/TA  0.181684